<a href="https://www.kaggle.com/code/dagartallison/parkinsons-submission-v1?scriptVersionId=127377910" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import sys
sys.path.append('/kaggle/input/amp-pd')

In [2]:
import pandas as pd
import numpy as np
import pickle
import amp_pd_peptide
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [3]:
def preprocess_train_df(train_clin_df, train_prot_df, train_pep_df):
    '''
        Takes in the train_clinical_data.csv, train_peptides.csv, train_proteins.csv as pandas dataframes
        Combines the protein and peptide data names and the joins with the train clinical data
        The dataframes are stratified kfold based on the target
        The function creates one dataframe for each target (updrs_1, updrs_2, updrs_3, updrs_4) stored in the final_df dictionary
        Returns a dictionary of the dataframes for each updrs target
    '''
    
    # drop the medication column
    train_clin_df = train_clin_df.drop(columns=['upd23b_clinical_state_on_medication'])
    
    # create a column with the UniProt and Peptide name combined
    train_pep_df['peptide_uniprot'] = train_pep_df['Peptide'] + '_'+ train_pep_df['UniProt']

    # create a table with the visit_id as the index and the proteins or peptides as the feature and the abundance as the values
    train_prot_pivot = train_prot_df.pivot(index='visit_id', values='NPX', columns='UniProt')
    train_pep_pivot = train_pep_df.pivot(index='visit_id', values='PeptideAbundance', columns='peptide_uniprot')

    # combine the two tables on the visit_id
    full_prot_train_df = train_prot_pivot.join(train_pep_pivot)

    # fill nan with 0 for this first round
    full_prot_train_df = full_prot_train_df.fillna(0)

    full_train_df = train_clin_df.merge(full_prot_train_df, how='inner', left_on='visit_id', right_on='visit_id')
    full_train_df = full_train_df.sample(frac=1).reset_index(drop=True)

    
    updrs = ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']

    final_dfs = dict()

    for target in updrs:
    
        to_remove = [updr for updr in updrs if updr != target]
        
        temp_train_df = full_train_df.drop(to_remove, axis=1)
        temp_train_df = temp_train_df.dropna()
        
        # calculate the number of bins by Sturge's rule
        num_bins = int(np.floor(1 + np.log2(len(full_train_df))))
        temp_train_df.loc[:, "bins"] = pd.cut(temp_train_df[target], bins=num_bins, labels=False)

        temp_train_df = temp_train_df.dropna().reset_index(drop=True)
        
        # initiate the kfold class from sklearn
        kf = StratifiedKFold(n_splits=5)
        
        # create a kfold column
        temp_train_df['kfold'] = -1

        # fill the kfold column
        for f, (t_, v_) in enumerate(kf.split(X=temp_train_df, y=temp_train_df['bins'].values)):
            temp_train_df.loc[v_, 'kfold'] = f
            
        # drop the bins column
        temp_train_df = temp_train_df.drop('bins', axis=1)
        
     

        final_dfs[target] = temp_train_df
            
    return final_dfs

In [4]:
def smape(y_true, y_pred):

    return round(np.mean(np.abs(y_pred - y_true) / ((np.abs(y_true) + np.abs(y_pred))/2)) * 100, 2)



def train_rf_model(df_dict):
    '''
        Takes in the preprocesses training dictionary of dataframes 
        Then trains a random forest regressor model on the data
        Returns a dictionary of models, one for each updrs target
    '''
    model_dict = {}
    
    updr1_model = RandomForestRegressor(random_state = 42)
    updr2_model = RandomForestRegressor(random_state = 42)
    updr3_model = RandomForestRegressor(random_state = 42)
    updr4_model = RandomForestRegressor(random_state = 42)
    
    for updr in ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']:
        df = df_dict[updr]
        df = df.drop(columns=['visit_id', 'patient_id', 'kfold'])


        x_train = df.drop(columns=[updr]).values
        y_train = df[updr].values

        
        if updr == 'updrs_1':
            updr1_model.fit(x_train, y_train)
            preds = updr1_model.predict(x_train)
            r2 = metrics.r2_score(y_train, preds)
            mape = metrics.mean_absolute_percentage_error(y_train, preds)
            s_mape = smape(y_train, preds)
            model_dict[updr] = updr1_model
            print(f'SMAPE = {s_mape}, R2 = {r2}, MAPE = {mape}')

        elif updr == 'updrs_2':
            updr2_model.fit(x_train, y_train)
            preds = updr2_model.predict(x_train)
            r2 = metrics.r2_score(y_train, preds)
            mape = metrics.mean_absolute_percentage_error(y_train, preds)
            s_mape = smape(y_train, preds)
            model_dict[updr] = updr2_model
            print(f'SMAPE = {s_mape}, R2 = {r2}, MAPE = {mape}')

        elif updr == 'updrs_3':
            updr3_model.fit(x_train, y_train)
            preds = updr3_model.predict(x_train)
            r2 = metrics.r2_score(y_train, preds)
            mape = metrics.mean_absolute_percentage_error(y_train, preds)
            s_mape = smape(y_train, preds)
            model_dict[updr] = updr3_model
            print(f'SMAPE = {s_mape}, R2 = {r2}, MAPE = {mape}')
        else:
            updr4_model.fit(x_train, y_train)
            preds = updr4_model.predict(x_train)
            r2 = metrics.r2_score(y_train, preds)
            mape = metrics.mean_absolute_percentage_error(y_train, preds)
            s_mape = smape(y_train, preds)
            model_dict[updr] = updr4_model
            print(f'SMAPE = {s_mape}, R2 = {r2}, MAPE = {mape}')
        
    
    return model_dict


## Create a Random Forest Model for each UPDRs

In [5]:
# read the training data with folds
train_df = pd.read_csv('/kaggle/input/amp-pd/train_clinical_data.csv')
train_prot_df = pd.read_csv('/kaggle/input/amp-pd/train_proteins.csv')
train_pep_df = pd.read_csv('/kaggle/input/amp-pd/train_peptides.csv')

train_df_dict = preprocess_train_df(train_df, train_prot_df, train_pep_df)

    
trained_models_dict = train_rf_model(train_df_dict)
    


SMAPE = 37.54, R2 = 0.8850159924543403, MAPE = 593016119472952.5
SMAPE = 67.49, R2 = 0.8964899496683623, MAPE = 1904457583997506.8
SMAPE = 58.8, R2 = 0.8969248525513086, MAPE = 4026550090282205.5
SMAPE = 129.74, R2 = 0.862884026356577, MAPE = 1955781138705183.5


## Create a Model for Forecasting the future months

In [6]:
def preprocess_forecast_train_df(preprocessed_train_df, target):
    '''
        Takes in the preprocessed training dataframe for a single updrs
        Returns a dataframe for forecasting which has columns for the updrs of different future visits
    '''
    
    temp_df = preprocessed_train_df[['visit_id', 'patient_id', target, 'visit_month']].sort_values(by=['patient_id', 'visit_month']).reset_index(drop=True)
    temp_pivot = temp_df.pivot(columns='visit_month', values=target, index='patient_id')
    temp_pivot = temp_pivot.reset_index()
    
    cols = [f'{target}_{month}' for month in temp_pivot.columns[1:]]
    temp_pivot.columns = ['patient_id'] + cols
    
    forecast_final = preprocessed_train_df[preprocessed_train_df['visit_month'] == 0]
    
    final_df = forecast_final.merge(temp_pivot, on=['patient_id'], how='left')
    
    final_df = final_df.drop(columns=['patient_id', target])
    if 'kfold' in final_df.columns:
        final_df = final_df.drop(columns=['kfold'])
        
    
    return final_df

### Create the forecasting dataframes

In [7]:
processed_forecast_dict = dict()

for target in ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']:
    
    forecast_data = train_df_dict[target]
    processed_forecast_dict[target] = preprocess_forecast_train_df(forecast_data, target)

In [8]:
def train_forecast(model, processed_forecast_dict, target, month_diff):
    
    # results dictionary for the models
    forecast_model_dict = dict()
    
    # get the training dataset
    df = processed_forecast_dict[target]
    
    forecast_cols = [col for col in df.columns if 'updrs' in col]

    drop_cols = [col for col in forecast_cols if col not in  [f'{target}_0', f'{target}_{month_diff}']]

    df = df.drop(columns=drop_cols)
    df = df.drop(columns=['visit_id', 'visit_month'])
    
    target_mo = f'{target}_{month_diff}'
    # drop nan rows for target column
    df = df.dropna(subset=[target_mo])
    
    x_train, x_valid, y_train, y_valid = train_test_split(df, df[target_mo], test_size=0.2, random_state=42)
    
    x_train = x_train.drop([target_mo], axis=1).values
    x_valid = x_valid.drop([target_mo], axis=1).values

    reg = model
    reg.fit(x_train, y_train)
    preds = reg.predict(x_valid)
    
    r2 = metrics.r2_score(y_valid, preds)
    mape = metrics.mean_absolute_percentage_error(y_valid, preds)
    s_mape = smape(y_valid, preds)
    
    print(target, month_diff, 'SMAPE:', s_mape)
    
    return model

In [9]:
forecast_dict = dict()
forecast_dict = {'updrs_1':{6:_, 12:_, 24:_}, 'updrs_2':{6:_, 12:_, 24:_}, 'updrs_3':{6:_, 12:_, 24:_}, 'updrs_4':{6:_, 12:_, 24:_}}

models = {'rf_reg': RandomForestRegressor(random_state=42)}

for updr in ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']:
    for month_diff in [6, 12, 24]:
        try:
            forecast_model = train_forecast(models['rf_reg'], processed_forecast_dict, updr, month_diff)
            forecast_dict[updr][month_diff] = forecast_model
        except:
            print(f'{updr} {month_diff} forecasting model failed!!!!!')

updrs_1 6 SMAPE: 76.09
updrs_1 12 SMAPE: 59.72
updrs_1 24 SMAPE: 71.5
updrs_2 6 SMAPE: 52.76
updrs_2 12 SMAPE: 78.11
updrs_2 24 SMAPE: 78.57
updrs_3 6 SMAPE: 33.83
updrs_3 12 SMAPE: 62.06
updrs_3 24 SMAPE: 76.03
updrs_4 6 forecasting model failed!!!!!
updrs_4 12 SMAPE: 119.38
updrs_4 24 SMAPE: 144.85


In [12]:
def preprocess_test_df(test_clin_df, test_prot_df, test_pep_df):
    '''
        Takes in the test data from the csv file in the form of a pandas dataframe
        Combines the peptide and protein data
        Outputs the dataframe for inference
    '''
    
    
    if 'upd23b_clinical_state_on_medication' in test_clin_df.columns:
        # drop the medication column
        test_clin_df = test_clin_df.drop(columns=['upd23b_clinical_state_on_medication'])
    
    if 'group_key' in test_clin_df.columns:
        # drop the group key
        test_clin_df = test_clin_df.drop(columns=['group_key'])
    
    # create a column with the UniProt and Peptide name combined
    test_pep_df['peptide_uniprot'] = test_pep_df['Peptide'] + '_'+ test_pep_df['UniProt']

    # create a table with the visit_id as the index and the proteins or peptides as the feature and the abundance as the values
    test_prot_pivot = test_prot_df.pivot(index='visit_id', values='NPX', columns='UniProt')
    test_pep_pivot = test_pep_df.pivot(index='visit_id', values='PeptideAbundance', columns='peptide_uniprot')

    # combine the two tables on the visit_id
    full_prot_test_df = test_prot_pivot.join(test_pep_pivot)

    # fill nan with 0 
    full_prot_test_df = full_prot_test_df.fillna(0)

    full_test_df = test_clin_df.merge(full_prot_test_df, how='inner', left_on='visit_id', right_on='visit_id')
    full_test_df = full_test_df.sample(frac=1).reset_index(drop=True)

    missing_row_id = [x for x in test_clin_df['row_id'] if x not in full_test_df['row_id'].to_list()]
    filtered_df = test_clin_df[test_clin_df['row_id'].isin(missing_row_id)]
    imputed_df = filtered_df.drop(columns=['visit_month']).merge(full_test_df.drop(columns=['row_id', 'visit_id']),
                                                                    how='left', 
                                                                    left_on=['patient_id', 'updrs_test'],
                                                                    right_on=['patient_id', 'updrs_test'])
    full_test_df = pd.concat([full_test_df, imputed_df])
    
    full_test_df = full_test_df.reset_index(drop=True)
    
    # remove the imputed visit month and replace from the test df
    full_test_df = full_test_df.drop(columns='visit_month').merge(test_df[['row_id', 'visit_month']], how='left', left_on='row_id', right_on='row_id')
    
    return full_test_df

In [13]:
test_df = pd.read_csv('/kaggle/input/amp-pd/example_test_files/test.csv')
test_prot_df = pd.read_csv('/kaggle/input/amp-pd/example_test_files/test_proteins.csv')
test_pep_df = pd.read_csv('/kaggle/input/amp-pd/example_test_files/test_peptides.csv')

test_preprocessed_df = preprocess_test_df(test_df, test_prot_df, test_pep_df)

In [14]:
test_df

,visit_id,visit_month,patient_id,updrs_test,row_id,group_key
0,3342_0,0,3342,updrs_1,3342_0_updrs_1,0
1,3342_0,0,3342,updrs_2,3342_0_updrs_2,0
2,3342_0,0,3342,updrs_3,3342_0_updrs_3,0
3,3342_0,0,3342,updrs_4,3342_0_updrs_4,0
4,50423_0,0,50423,updrs_1,50423_0_updrs_1,0
5,50423_0,0,50423,updrs_2,50423_0_updrs_2,0
6,50423_0,0,50423,updrs_3,50423_0_updrs_3,0
7,50423_0,0,50423,updrs_4,50423_0_updrs_4,0
8,3342_6,6,3342,updrs_1,3342_6_updrs_1,6
9,3342_6,6,3342,updrs_2,3342_6_updrs_2,6


In [15]:
test_preprocessed_df.sort_values(by='visit_month')

,visit_id,patient_id,updrs_test,row_id,O00391,O00533,O00584,O14498,O14773,O14791,...,YTTEIIK_P00736,YVGGQEHFAHLLILR_P02763,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR_P00738,YVMLPVADQDQC(UniMod_4)IR_P00738,YVNKEIQNAVNGVK_P10909,YWGVASFLQK_P02753,YYC(UniMod_4)FQGNQFLR_P02790,YYTYLIMNK_P01024,YYWGGQYTWDMAK_P02675,visit_month
0,50423_0,50423,updrs_3,50423_0_updrs_3,33127.9,490742.0,43615.3,0.0,16486.6,2882.42,...,15370.0,4137590.0,15217.0,83253.9,110941.0,112251.0,221372.0,46557.4,8527.23,0
1,50423_0,50423,updrs_1,50423_0_updrs_1,33127.9,490742.0,43615.3,0.0,16486.6,2882.42,...,15370.0,4137590.0,15217.0,83253.9,110941.0,112251.0,221372.0,46557.4,8527.23,0
4,50423_0,50423,updrs_4,50423_0_updrs_4,33127.9,490742.0,43615.3,0.0,16486.6,2882.42,...,15370.0,4137590.0,15217.0,83253.9,110941.0,112251.0,221372.0,46557.4,8527.23,0
5,50423_0,50423,updrs_2,50423_0_updrs_2,33127.9,490742.0,43615.3,0.0,16486.6,2882.42,...,15370.0,4137590.0,15217.0,83253.9,110941.0,112251.0,221372.0,46557.4,8527.23,0
8,3342_0,3342,updrs_1,3342_0_updrs_1,14564.0,616804.0,61418.1,27830.1,14959.2,2853.12,...,10359.8,3657330.0,47100.5,1007150.0,74598.6,127126.0,210763.0,46207.2,22031.60,0
9,3342_0,3342,updrs_2,3342_0_updrs_2,14564.0,616804.0,61418.1,27830.1,14959.2,2853.12,...,10359.8,3657330.0,47100.5,1007150.0,74598.6,127126.0,210763.0,46207.2,22031.60,0
10,3342_0,3342,updrs_3,3342_0_updrs_3,14564.0,616804.0,61418.1,27830.1,14959.2,2853.12,...,10359.8,3657330.0,47100.5,1007150.0,74598.6,127126.0,210763.0,46207.2,22031.60,0
11,3342_0,3342,updrs_4,3342_0_updrs_4,14564.0,616804.0,61418.1,27830.1,14959.2,2853.12,...,10359.8,3657330.0,47100.5,1007150.0,74598.6,127126.0,210763.0,46207.2,22031.60,0
2,3342_6,3342,updrs_4,3342_6_updrs_4,14564.0,616804.0,61418.1,27830.1,14959.2,2853.12,...,10359.8,3657330.0,47100.5,1007150.0,74598.6,127126.0,210763.0,46207.2,22031.60,6
3,3342_6,3342,updrs_1,3342_6_updrs_1,14564.0,616804.0,61418.1,27830.1,14959.2,2853.12,...,10359.8,3657330.0,47100.5,1007150.0,74598.6,127126.0,210763.0,46207.2,22031.60,6


In [18]:

def prepare_model_df(model_df, target, train_df , visit_month=0):
    
    pred_cols = [col for col in train_df.columns if col not in ['visit_id', 'patient_id', target, 'kfold']]

    # add visit_month if it is not in the model_df.columns
    if 'visit_month' not in model_df.columns:
        model_df['visit_month'] = visit_month
    
    model_df = model_df[model_df['visit_month'] == visit_month]
    
    model_df = model_df[model_df['updrs_test'] == target]
    
    # find the columns in preds_cols that are not in the model_df.columns
    not_in_pred_cols = [col for col in pred_cols if col not in model_df.columns]

    # create an empty dataframe with the columns in not_in_pred_cols
    not_in_preds_df = pd.DataFrame(columns=not_in_pred_cols)

    # combine the model_df and the not_in_preds_df so all the needed columns are in dataframe
    new_model_df = pd.concat([model_df, not_in_preds_df], axis=1)
    
    # fill the nan values with 0
    new_model_df = new_model_df.fillna(0)
    
    row_id_df = new_model_df['row_id']

    # filter the new_model_df to only include the columns in pred_cols with the correct order
    return new_model_df[pred_cols], row_id_df

In [19]:
md_df, row_id = prepare_model_df(test_preprocessed_df, 'updrs_1', train_df_dict['updrs_1'], visit_month=0)

In [29]:
target = 'updrs_4'
month_diff = 12
# get the training dataset
df = processed_forecast_dict[target]
forecast_cols = [col for col in df.columns if 'updrs' in col]
drop_cols = [col for col in forecast_cols if col not in  [f'{target}_0', f'{target}_{month_diff}']]

df = df.drop(columns=drop_cols)
df = df.drop(columns=['visit_id', 'visit_month'])
target_mo = f'{target}_{month_diff}'

# drop nan rows for target column
df = df.dropna(subset=[target_mo])



,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,O43505,O60888,...,YVGGQEHFAHLLILR_P02763,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR_P00738,YVMLPVADQDQC(UniMod_4)IR_P00738,YVNKEIQNAVNGVK_P10909,YWGVASFLQK_P02753,YYC(UniMod_4)FQGNQFLR_P02790,YYTYLIMNK_P01024,YYWGGQYTWDMAK_P02675,updrs_4_0,updrs_4_12
1,0.00,409011.0,25629.10,0.0,14566.30,0.00,105005.0,35929.70,146784.0,96839.2,...,3173510.0,0.00,154088.0,67545.0,76966.50,481859.0,38969.3,1114.63,0.0,3.0
3,13541.60,421862.0,27429.90,21302.7,21044.50,1786.04,110972.0,62374.60,177221.0,230591.0,...,5231670.0,0.00,1231820.0,105571.0,161194.00,725132.0,66307.7,28106.90,0.0,0.0
4,0.00,410448.0,19021.20,25201.5,20097.40,2752.37,87763.1,40135.00,148991.0,97387.6,...,3162940.0,24405.30,132960.0,113977.0,102673.00,425555.0,30487.5,19667.30,0.0,0.0
5,9811.48,763519.0,26315.20,23847.3,4457.76,3675.85,129371.0,69984.70,241531.0,190487.0,...,4565910.0,19756.20,128486.0,105367.0,126974.00,294284.0,51827.3,23800.60,0.0,0.0
6,0.00,478938.0,6437.41,17809.8,12897.20,2149.96,133727.0,53822.80,214889.0,121960.0,...,2697150.0,48132.00,309020.0,73330.3,151379.00,570099.0,48505.1,17111.90,2.0,2.0
7,0.00,122737.0,11237.60,0.0,9081.72,0.00,17571.6,5806.84,65130.6,83339.0,...,4749580.0,103859.00,493403.0,27703.4,0.00,363865.0,28884.6,0.00,0.0,0.0
8,12954.10,540854.0,21076.50,26982.5,26061.10,3371.23,151636.0,69226.20,290515.0,142652.0,...,3061410.0,84719.40,908722.0,130729.0,133948.00,389727.0,70281.0,31245.70,9.0,9.0
9,0.00,415929.0,16889.70,0.0,12339.60,0.00,123464.0,39411.00,145834.0,148370.0,...,3681930.0,43625.30,521477.0,58612.1,60749.70,496057.0,43116.0,2433.70,6.0,7.0
12,12064.20,660296.0,26954.30,26432.2,19623.20,0.00,154358.0,60022.10,215730.0,96329.4,...,3895330.0,110553.00,625162.0,134878.0,155797.00,463070.0,50224.3,0.00,0.0,0.0
14,0.00,322915.0,0.00,40920.1,19994.00,2266.96,177311.0,60117.40,169871.0,105004.0,...,3188420.0,0.00,352419.0,156569.0,121983.00,507854.0,0.0,35345.60,3.0,3.0


In [ ]:
prepare_forecast_model_df(updr_df, 'updrs_1',processed_forecast_dict )

In [ ]:
def prepare_forecast_model_df(model_df, target, processed_forecast_dict , visit_month=6):
    
    df = processed_forecast_dict[target]
    forecast_cols = [col for col in df.columns if 'updrs' in col]
    drop_cols = [col for col in forecast_cols if col not in [f'{target}_0', f'{target}_{month_diff}']]

    # add visit_month if it is not in the model_df.columns
    if 'visit_month' not in model_df.columns:
        model_df['visit_month'] = visit_month
    
  
    print(drop_cols)
#     model_df = model_df[model_df['visit_month'] == visit_month]
    
#     model_df = model_df[model_df['updrs_test'] == target]
    
#     # find the columns in preds_cols that are not in the model_df.columns
#     not_in_pred_cols = [col for col in pred_cols if col not in model_df.columns]

#     # create an empty dataframe with the columns in not_in_pred_cols
#     not_in_preds_df = pd.DataFrame(columns=not_in_pred_cols)

#     # combine the model_df and the not_in_preds_df so all the needed columns are in dataframe
#     new_model_df = pd.concat([model_df, not_in_preds_df], axis=1)
    
#     # fill the nan values with 0
#     new_model_df = new_model_df.fillna(0)

#     # filter the new_model_df to only include the columns in pred_cols with the correct order
#     return new_model_df[pred_cols]

In [ ]:
full_test_df = preprocess_test_df(test_df, test_prot_df, test_pep_df)

model_df = prepare_model_df(full_test_df, 'updrs_1',train_df_dict['updrs_1'], visit_month=0)
model_df

## test out the submission code


In [ ]:

full_test_df = preprocess_test_df(test_df, test_prot_df, test_pep_df)
full_test_df['visit_month'] = full_test_df['visit_id'].str.extract(r'\d+_([0-9]+)').astype(int)

final_pred_df = pd.DataFrame()

for updr in ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']:
    updr_df = full_test_df[full_test_df['updrs_test'] == updr]
    info_cols = ['visit_id', 'visit_month', 'patient_id', 'updrs_test', 'row_id']


    for month in [0, 12, 24]:
            
        if month == 0:
            # prepare the model_df for the correct month
            model_df = prepare_model_df(updr_df, updr, train_df_dict[updr], visit_month=month)

            updr_info = updr_df[updr_df['visit_month'] == month]
            updr_info = updr_info[info_cols]

            # Load the saved model from file
            rf_reg = trained_models_dict[updr][month]

            # Use the imported model to make predictions
            y_pred = rf_reg.predict(model_df.values)
                
            updr_info[f'plus_{month}_months'] = y_pred
                
        elif month in [12, 14]:
            
            # prepare the model_df for the correct month
            model_df = prepare_model_df(updr_df, updr, processed_forecast_dict[updr], visit_month=month)

            updr_info = updr_df[updr_df['visit_month'] == month]
            updr_info = updr_info[info_cols]

            # Load the saved model from file
            rf_reg = forecast_dict[updr][month]

            # Use the imported model to make predictions
            y_pred = rf_reg.predict(model_df.values)
                
            updr_info[f'plus_{month}_months'] = y_pred
            
        

In [ ]:
updr_info

In [ ]:


def create_submission_df(test_df, prot_test_df, pep_test_df, train_df):
    
    full_test_df = preprocess_test_df(test_df, prot_test_df, pep_test_df)

    final_pred_df = pd.DataFrame()

    for updr in ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']:
        updr_df = full_test_df[full_test_df['updrs_test'] == updr]
        info_cols = ['visit_id', 'visit_month', 'patient_id', 'updrs_test', 'row_id']
        updr_info = updr_df[info_cols]
        model_df = updr_df.drop(columns=info_cols)
        

        for month in [0, 6, 12, 24]:
            
            if month == 0:
                # prepare the model_df for the correct month
                model_df = prepare_model_df(model_df, updr, train_df[updr], visit_month=month)
            
                # Load the saved model from file
                rf_reg = forecast_dict[updr][month]

                # Use the imported model to make predictions
                y_pred = rf_reg.predict(model_df.values)
                
                updr_info[f'plus_{month}_months'] = y_pred
            
            # temporary fix for missing model
            elif updr == 'updrs_4' and month == 6:
                
                # prepare the model_df for the correct month
                model_df = prepare_model_df(model_df, updr, train_df[updr], visit_month=0)

                # Load the saved model from file
                rf_reg = trained_models[updr]

                # Use the imported model to make predictions
                y_pred_1 = rf_reg.predict(model_df.values)
                
        
                # prepare the model_df for the correct month
                model_df = prepare_model_df(model_df, updr, train_df[updr], visit_month=12)
                # Load the saved model from file
                rf_reg = trained_models[updr]

                # Use the imported model to make predictions
                y_pred_2 = rf_reg.predict(model_df.values)
                
                y_pred = (y_pred_1 + y_pred_2) / len(y_pred_1)
                
                
                updr_info[f'plus_{month}_months'] = y_pred
                
                
            else:
                # prepare the model_df for the correct month
                model_df = prepare_model_df(model_df, updr, train_df[updr], visit_month=month)
            
                # Load the saved model from file
                rf_reg = forecast_dict[updr][month]

                # Use the imported model to make predictions
                y_pred = rf_reg.predict(model_df.values)
                
                updr_info[f'plus_{month}_months'] = y_pred
        
        
        final_pred_df = pd.concat([final_pred_df, updr_info])
        
    submit_df = pd.DataFrame(columns=['prediction_id', 'rating'])

    for i, row in final_pred_df.iterrows():
        for col in ['plus_0_months', 'plus_6_months', 'plus_12_months', 'plus_24_months']:
            submit_df = submit_df.append({'prediction_id': row['row_id']+'_'+col, 'rating': row[col]}, ignore_index=True)
            
    submit_df['rating'] = submit_df['rating'].astype(int)

    
    return submit_df

## Simple Linear Regression Model for testing how to Submit via the API

In [ ]:
model = {}
target = ["updrs_1", "updrs_2", "updrs_3", "updrs_4"]
train = pd.read_csv("/kaggle/input/amp-pd/train_clinical_data.csv")


for u in target:
        
    # Drop NAs
    temp = train.dropna(subset=[u]) 
    
    # Train data
    X = temp['visit_month']
    y = temp[u]
        
    trained = LinearRegression().fit(X.values.reshape(-1, 1), y)
    
    # Save model
    model[u] = trained

In [ ]:
def get_predictions(my_train, pro, model):

    # Forecast
    my_train = my_train.fillna(0)
    
    for u in target:
        
        # Here is where we will save the final results
        my_train['result_' + str(u)] = 0
  
        # Predict    
        X = my_train["visit_month"]
        
        if u == 'updrs_4':
            my_train['result_' + str(u)] = 0
        else:
            my_train['result_' + str(u)] = np.ceil(model[u].predict(X.values.reshape(-1, 1)))

        
    # Format for final submission
    result = pd.DataFrame()

    for m in [0, 6, 12, 24]:
        for u in [1, 2, 3, 4]:

            temp = my_train[["visit_id", "result_updrs_" + str(u)]]
            temp["prediction_id"] = temp["visit_id"] + "_updrs_" + str(u) + "_plus_" + str(m) + "_months"
            temp["rating"] = temp["result_updrs_" + str(u)]
            temp = temp [['prediction_id', 'rating']]

            result = result.append(temp)            
    result = result.drop_duplicates(subset=['prediction_id', 'rating'])

    return result

# Run once to check results
get_predictions(train, None, model)

In [ ]:
test_df = pd.read_csv('/kaggle/input/amp-pd/example_test_files/test.csv')
test_prot_df = pd.read_csv('/kaggle/input/amp-pd/example_test_files/test_proteins.csv')
test_pep_df = pd.read_csv('/kaggle/input/amp-pd/example_test_files/test_peptides.csv')

train_df = pd.read_csv('/kaggle/input/amp-pd/train_clinical_data.csv')
train_prot_df = pd.read_csv('/kaggle/input/amp-pd/train_proteins.csv')
train_pep_df = pd.read_csv('/kaggle/input/amp-pd/train_peptides.csv')

# Run once to check results
lr_preds = get_predictions(test_df, test_prot_df, model)

my_preds = create_submission_df(test_df,test_prot_df, test_pep_df)


In [ ]:
test_df.columns, train_df.columns

In [ ]:
[updr for updr in lr_preds['prediction_id']if updr not in my_preds['prediction_id'].to_list()]

## Submission Through the API

In [ ]:
env = amp_pd_peptide.make_env()   # initialize the environment
iter_test = env.iter_test()  

In [ ]:
for (test, test_peptides, test_proteins, sample_submission) in iter_test:
        
    result = create_submission_df(test, test_proteins, test_pep_df)

    env.predict(result)   # register your predictions

